## For BA to solve the Non-linear optimization problem of SLAM, the following need to exist:
1. Corresponding observations among selected keyframes.
2. A method of keyframe selection; avoid unnecessay redundancy.
3. A strong network of keyframes observing points with significant parallax.
4. Initial estimate of keyframe poses and point locations.
5. Optimization focussed in a local map; to provide scalability.
6. Fast global optimization for real time loop closure.

## Main Features of ORB-SLAM:
1. Using **_same_** features (ORB) for __Tracking__, __Mapping__, __Reloc__ and __Loop Closure__; efficient and reliable.
2. __Survival of fittest__ approach for keyframe selection; generous spawing, restrictive culling.
* __Essential Graph__; loop closure links, strong edges from covisibility graph; real time loop closing.
* Automatic and robus initialization; model selection, initial map from planar and non-planar scenes.
* __Covisibility Graph__ for tracking and mapping in a local covisible area; real time operation.
* Real time __Camera Relocalization__; allows better recovery from tracking failure, enhances map reuse.


## Optimization Objective:
1. Non-linear optimization of **Feature Reprojection Errors**.


## System Overview
__A. Feature Choice:__
  * Same features for mapping, tracking, reloc and loop closure.
  * Features need to be extracted in < 33ms; can;t do SIFT, SURF, A-KAZE
  * Need to be rotation invariant to support place recog; can't do BRIEF, LDB
  * Use ORB (Oriented multi-scale FAST, 256-bit descriptor
    + Extremely fast
    + Viewpoint invariant

__B. 3 Threads; Tracking, Local Mapping and Loop Closure:__
  1. Tracking thread:
    * Localization of camera with every frame, __decide if keyframe__
    * Feature matching with previous frame, optimization with motion-only BA.
    * If tracking is lost, place recog module used for global reloc.
    * Once there is an initial estimate of the camera pose and feature matching, covisibility graph used to get a local visible map.
    * Search for matches in local map by reprojection; optimize camera pose again.
    * Decide if new keyframe is to be inserted<br><br>
  2. Local Mapping thread:
    * Process new keyframes, perform local BA to get optimal reconstruction around camera pose.
    * Find correspondences for the new unmatched ORB features in connected keyframes in the covisibility graph, triangulate new points.
    * Point removal to retain high quality points.
    * Redundant Keyframe culling.<br><br>
  3. Loop Closure thread:
    * Search for loop closure with every new keyframe.
    * If loop detected, calculate similarity transform, (measure of drift accumulated in the loop).
    * Align loops, fuse duplicate points.
    * Pose graph optimization of __Essential Graph (sparser subset of covisibility graph)__
    
__C. Map points, keyframes and how to select:__
  * *Each map point* $p_i$ stores:   
    1. 3-D position in World Coordinate system: $X_{w,i}$  
    2. Viewing direction: $n_i$, mean unit vector of all vieweing directions (rays that join point to optical center of observer keyframe)
    3. ORB descriptor: $D_i$, minimum hamming distance out of all other descriptors from all the observer keyframes
    4. Max and min distance at which a keypoint can be observed: $d_{max}$, $d_{min}$, wrt scale invariance limits of ORB<br><br>
  
  * *Each keyframe* $K_i$ stores:
    1. Camera Pose: $T_{iw}$, rigid body transformation from World to Camera. Transformation of points from world to camera.
    2. Camera intrinsics (including focal length and principal point).
    3. All ORB featues extracted in the frame (undistored co-ordinates if distortion model provided).
    
__D. Covisibility Graph and Essential Graph:__
  * Covisibility represented as an *undirected weighted* graph.
    * Each node is a keyframe.
    * Number of common map points, $\theta$ is the weight of the edge between 2 keyframes. For an edge to exist, KFs should share observations of same (atleast 15) map points.<br><br>
  * Essential Graph:
    * Retains all nodes (keyframes) of Covisibilty graph but fewer edges; still preserves a strong network.
    * Incrementally built spanning tree; connected subgraph of covisibility graph with minimum number of edges.
    * High Covisibility edges are edges with $\theta_{min} = 100$

__E. Bag of Words Place Recog:__
  * Incrementally update database of Invert Index. For each *visual word* in the vocab, we store the keyframe index where it was observed. Faster querying.
  * Keyframes are grouped by connections in the Covisibility Graph.
  * Multiple candidates returned with score > 0.75% of best score.
  * This BoW embedding is also used for fidning correspondences. Brute force match constrained only to features that belong to same node of the vocab tree. Paper uses Second out of Six. Faster search.
      * Refine correspondences with orientation consistency check, coherent rotation for all correspondences.

## Automatic Map Init
__* Goal: To compute relative pose between two frames to traingulate an inital set of map points.*__<br>
__* Should be scene independent (planar or non planar)*__<br>
__* should not require human intervention.*__
* Compute 2 geometric models. Use _a heuristic metric_ to select best model.
    * **Homography**, assuming planar scene
    * **Fundamental matrix**, assuming general (non planar) scene
* Steps:
    1. Find initial correspondences:
        * Extract ORB feature (finest scale only) in Current Frame $F_c$ and references in Reference Frame $F_r$, $x_c \leftrightarrow x_r$.
        * If not enough matches, reset $F_r$
    2. Parallel computuation of 2 models:
        * For homogeneity between models, same RANSAC scheme for both, same # of iterations, same set of points for both models at each iteration.
        * Homography $H_{cr}$
            * Normalized DLT for Homography, $x_c = H_{cr}x_r$
            * 4 point correspondences
        * Fundamental Matrix $F_{cr}$
            * Eight-point algorithm, ${x_c}^T F_{cr} x_r = 0$
            * 8 correspondences, 4 of which are also used for Homography.
        * Compute a score $S_M$ for each models (M is H and F)
        * Keep homography AND fundamental matrix with highest score. We will then select in next step.
    3. Model Selection:
        * Calculate $R_H = \large\frac{S_H}{S_H + S_F}$
        * Select homography if $R_H > 0.45$, else selct fundamental matrix
    4. Motion and SfM recovery:
        * Calculate motion hypothesis from the model selected in 3.
        * If Homography, we get 8 motion hypothesis.
            * Triangulate and select best reconstruction.
        * If Fundamental Matrix:
            * Get Essential Matrix, $E_{rc} = K^T F_{rc} K$
            * Get 4 motion hypothesis with SVD
            * Triangulate and select best reconstruction.
    5. Bundle Adjustment:
        * Full BA to refine initial reconstruction

## Tracking
__* Done for every frame *__<br>
__* Camera pose optimization is Motion-Only BA *__<br>

**A. ORB Extraction:**
  * Extract FAST corners at eight scale levels with a scale factor of 1.2.
  * Compute ORB descriptors on the retained FAST corners.
  
**B. Initial pose estimation from previous frame:**
  * **If tracking was successful** for last frame, use constant velocity model.
  * Predict camera pose and perform guided search of map points seen in last frame.
  * If not enough matches, the motion model is violated, use a wider search area.
  * Optimize pose with found correspondences.
  
**C. Initial pose estimation via Global Relocalization:**
  * **If tracking is lost**, convert frame to Bag of Words and query recognition DB for keyframe candidates for global reloc.
  * Computer correspondences with ORB  for map points in each keyframe.
  * Perform RANSAC with every keyframe and find camera pose using PnP.
  * If pose found with enough keyframes perform guided search for more matches of map points in the candidate keyframe.
  * Optimize camera pose.
  
**D. Track Local Map:**
  * We now have Initial camera pose estimate and an initial set of features.
  * Project the map into the frame and find more map point correspondences.
  * **Use local map** to reduce complexity in large maps.
  * The local map contains 3 types of Keyframes, $\large\kappa_i$
    * $\large\kappa_1$: set of keyframes that share map points with current frame
    * $\large\kappa_2$: neighbors of $\large\kappa_1$ keyframes from covisibility graph
    * $K_{ref}$: $K_{ref} \in \large\kappa_1$; keyframe with max number of shared map points with current frame
  * Each map point in $\large\kappa_1$ and $\large\kappa_2$ is searched in current frame like this:
    * Compute map point projection $\mathbf{x}$ in current frame. Discard if out of image bounds.
    * Compute angle between current viewing ray $\mathbf{v}$ and map point mean viewing direction $\mathbf{n}$. Discard if ${\mathbf{v}}\cdot{\mathbf{n}} < \cos(60 ^{\circ})$$d$
    * Compute distance $d$ from map point to camera center. Discard if outside scale invariance bounds $d \notin [d_{min}, d_{max}]$. System Overview 'C'
    * Computer Scale in frame as ratio $ \large\frac{d}{d_{min}}$
    * Compare the descriptor $$\mathbf{D}$ of the map point with the still unmatched ORB features in the frame, at the predicted scale, around $\mathbf{x}$ and associate amp point with the best match.
  * Optimize camera pose with all map points found in the frame.
  
**E. New Keyframe decision:**
  * Since we cull keyframes later, we add keyframes as fast as possible.
  * For the current frame to become a keyframe, following need to be true:
      1. More than 20 frames have passed since last global reloc. **Ensures good reloc**
      2. Local mapping is idle or more than 20 frames have passed since last keyframe creation.
      3. Current frame tracks atelast 50 points. **Ensures good tracking**
      4. Current frame tracks less than 90% points of $K_{ref}$. **Imposes a minimum visual change, instead of distance criterion from other keyframes**

## Local Mapping
__* Done for every new keyframe $K_i$ *__<br>

**A. KeyFrame Insertion:**
  * Update the Covisibility Graph:
      * Add new node for $K_i$
      * Upadte edges to other keyframes resulting from the shared map points
  * Update spanning tree
      * Link $K_i$ with the keyframe with most points in common.
  * Compute a BoW representation for $K_i$. This will help in data association for triangulating new points.
  
**B. Recent Map Point Culling:**
  * Test if a map point is keep-worthy during the first 3 keyframes after creation. Ensure a point is trackable and not wrongly triangulated.
  * A map point must fulfill 2 conditions to avoid culling:
      * Tracking should find a point in more than 25% of the frames in which it is predicted to be visible.
      * If more than one keyframe has passed since creation, it must be observed from at least 3 keyframes.
  * Once a map point passes a test, it can be removed only if it is observed from less than 3 keyframes.
      * This might happen if a keyframe is culled later on
      * Can also happen when local BA discards outlier observations.
      
**C. New Map Point Creation:**
  * Triangulate ORB features from connected keyframes $\large\kappa_c$ in the Covisibility graph.
      * Match unmatched ORB in $K_i$ with unmatched points in other $\large\kappa_c$ frames. Use the BoW for brute force pruning (III-E), then discard if epipolar constraint not satisfied.
      * Upon triangulation, to accept new points check for **positive depth in both cameras, parallax, reprojection error and scale consistency**.
      * A map point could be matched in multiple frames; project and search as in Tracking- D. Track local map
      
**D. Local Bundle Adjustment:**
  * Local BA optimization of $K_i$, all connected keyframes in covisibility graph $\large\kappa_c$ and all map points seen in those keyframes
  * All other keyframes that see those map points, but are not connected, are also added to optimization but act as anchors and remain fixed.
  * Outlier observations are discarded in middle and end of optimization.
  
**E. Local Keyframe Culling:**
  * Redundant keyframes need to be discarded to enable lifelong operation. we want keyframes to be added only when there is enough visual content change.
  * Remove all keyframes in $\large\kappa_c$ where more than 90% of its points can be seen by atleast 3 other keyframes in the same or finer scale. The finer the scale, the more accurate the depth information.